## Setup

In [1]:
!ln -s ./../lung_cancer_detection

ln: ./lung_cancer_detection: File exists


In [2]:
from pathlib import Path

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import wandb

from lung_cancer_detection.data.data_module import LIDCDataModule
from lung_cancer_detection.models.nodule_segmentation import NoduleSegmentationUNet
from lung_cancer_detection.experiment import Experiment
from lung_cancer_detection.utils import load_config

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
config_path = Path().absolute().parents[0] / "configs/test.yaml"
config_path.exists()

True

## Load configuration

In [5]:
config = load_config(config_path)
config

{'random_seed': 47,
 'wandb': {'offline': False,
  'project': 'lung-cancer-detection',
  'tags': ['nodule-segmentation', 'testing']},
 'artifacts': {'seg_data_artifact': {'name': 'lidc-idri-cts',
   'version': 'v0',
   'type': 'dataset',
   'description': 'Ten chest CT scans from LIDC-IDRI dataset including metadata'},
  'seg_model_artifact': {'name': 'nodule-segmentation-model',
   'version': 'v0',
   'type': 'model',
   'description': 'Downsized UNet for segmenting lung nodules in chest CTs'}},
 'data': {'raw_dir': '/Volumes/LaCie/data/lung-cancer-detection/lidc-idri/LIDC-IDRI/',
  'data_dir': 'data/test/',
  'cache_dir': 'data/cache/',
  'sample_size': False,
  'batch_size': 2,
  'val_split': 0.2},
 'seg_model': {'features': [4, 4, 8, 16, 32, 4],
  'dropout': 0.0,
  'lr': 0.0001,
  'norm': ['instance', {'affine': True}]},
 'experiment': {'max_epochs': 2,
  'auto_lr_find': True,
  'gpus': 0,
  'auto_select_gpus': False}}

In [6]:
config["data"]["data_dir"] = Path().absolute().parents[0] / config["data"]["data_dir"]
config["data"]["cache_dir"] = Path().absolute().parents[0] / config["data"]["cache_dir"]
print(config["data"]["data_dir"])
print(config["data"]["cache_dir"])
print(config["data"]["data_dir"].exists(), config["data"]["cache_dir"].exists())

/Users/felix/code/ml/lung-cancer-detection/data/test
/Users/felix/code/ml/lung-cancer-detection/data/cache
True True


## Create model and data modules

In [7]:
model = NoduleSegmentationUNet(**config["seg_model"])

BasicUNet features: (4, 4, 8, 16, 32, 4).


In [8]:
data = LIDCDataModule(**config["data"])

## Create logger

In [9]:
logger = WandbLogger(**config["wandb"], job_type="training")

## Create callbacks

In [10]:
es = EarlyStopping(monitor="val_loss", verbose=True)
mc = ModelCheckpoint(monitor="val_loss", filename="{epoch}-{step}-{val_loss:.4f}", verbose=True, save_top_k=1)
callbacks = [es, mc]

EarlyStopping mode set to min for monitoring val_loss.


## Create experiment

In [11]:
exp = Experiment(model, 
                 data, 
                 logger, 
                 input_artifact=config["artifacts"]["seg_data_artifact"],
                 callbacks=callbacks, **config["experiment"])

Global seed set to 47
wandb: Currently logged in as: felixpeters (use `wandb login --relogin` to force relogin)


GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [12]:
exp.run()


  | Name  | Type      | Params
------------------------------------
0 | model | BasicUNet | 90.5 K
1 | loss  | DiceLoss  | 0     
------------------------------------
90.5 K    Trainable params
0         Non-trainable params
90.5 K    Total params
0.362     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 3: val_loss reached 0.69889 (best 0.69889), saving model to "/Users/felix/code/ml/lung-cancer-detection/nbs/wandb/run-20210427_105528-3mrmskp5/files/lung-cancer-detection/3mrmskp5/checkpoints/epoch=0-step=3-val_loss=0.6989.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 7: val_loss reached 0.69810 (best 0.69810), saving model to "/Users/felix/code/ml/lung-cancer-detection/nbs/wandb/run-20210427_105528-3mrmskp5/files/lung-cancer-detection/3mrmskp5/checkpoints/epoch=1-step=7-val_loss=0.6981.ckpt" as top 1


In [13]:
output_artifact = config["artifacts"]["seg_model_artifact"]
output_artifact["path"] = Path(mc.best_model_path).absolute()

In [14]:
exp.finish(output_artifact)

In [15]:
wandb.finish()

train_loss_epoch,0.70166
epoch,1
trainer/global_step,7
_runtime,246
_timestamp,1619513974
_step,5
val_loss_step/epoch_0,0.69889
val_loss_epoch,0.6981
val_loss_step/epoch_1,0.6981


train_loss_epoch,█▁
epoch,▁▁██
trainer/global_step,▄▁▄█▁█
_runtime,▁▂▂▇██
_timestamp,▁▂▂▇██
_step,▁▂▄▅▇█
val_loss_step/epoch_0,▁
val_loss_epoch,█▁
val_loss_step/epoch_1,▁
